In [20]:
import pandas as pd

def clean_data(df_path):
  # convert datasets to pandas
  df = pd.read_csv(df_path)
  # clean data(drop any empty, drop duplicates)
  df.dropna(inplace=True)
  df.drop_duplicates(inplace=True)
  # convert to pandas timestamps
  df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
  print(df.dtypes)
  return df

In [ ]:
# # drop data that is not in illinois

# df = pd.read_csv('ActualEnergyPrice.csv')
# df.drop(df[df['HUB'] != 'ILLINOIS.HUB'].index, inplace=True)
# df.to_csv('ActualEnergyPrice.csv', index=False)
# df = pd.read_csv('DayAheadEnergyPrice.csv')
# df.drop(df[df['node'] != 'ILLINOIS.HUB'].index, inplace=True)
# df.to_csv('DayAheadEnergyPrice.csv', index=False)



FileNotFoundError: [Errno 2] No such file or directory: 'ActualEnergyPrice.csv'

In [22]:
# clean
# ActualEnergyPrice.csv
# DayAheadEnergyPrice.csv
# ActualLoad.csv
# ForecastedLoad.csv


# combining all the datasets by date

file_paths = [
    'ActualEnergyPrices2.csv',
    'DayAheadEnergyPrice2.csv',
    'ActualLoad2.csv',
    'ForecastedLoad2.csv'
]

dfs = []
for file_path in file_paths:
  df = clean_data(file_path)
  dfs.append(df)
# merging the data
merged_df = dfs[0]
for i in range(1, len(dfs)):
  merged_df = pd.merge(merged_df, dfs[i], on='Date', how='outer')

#renaming the columbs
new_column_names = {
    'LMP': 'actual_lmp',
    'lmp': 'day_ahead_lmp',
    'Load_x': 'actual_load',
    'Load_y': 'forecasted_load'
}
merged_df.rename(columns=new_column_names, inplace=True)
# cleaning the data for merged
merged_df.dropna(inplace=True)
merged_df.drop_duplicates(inplace=True)
merged_df.drop(columns=['node'], inplace=True)

merged_df.to_csv('merged_data.csv', index=False)
merged_df.to_pickle('merged_data.pkl')

Date    datetime64[ns]
LMP            float64
HUB             object
dtype: object
Date    datetime64[ns]
lmp            float64
node            object
dtype: object
Date    datetime64[ns]
Load             int64
dtype: object
Date    datetime64[ns]
Load             int64
dtype: object


In [26]:
print(merged_df.tail())

                       Date  actual_lmp           HUB  day_ahead_lmp  \
1599835 2025-02-05 18:00:00      125.79  ILLINOIS.HUB          38.51   
1599836 2025-02-05 18:00:00      125.79  ILLINOIS.HUB          47.05   
1599837 2025-02-05 18:00:00      125.79  ILLINOIS.HUB          40.18   
1599838 2025-02-05 18:00:00      125.79  ILLINOIS.HUB          36.19   
1599839 2025-02-05 18:00:00      125.79  ILLINOIS.HUB          36.92   

         actual_load  forecasted_load  AveragePrice_OtherStates  
1599835      85090.0          81441.0                 85.034286  
1599836      85090.0          81441.0                 85.034286  
1599837      85090.0          81441.0                 85.034286  
1599838      85090.0          81441.0                 85.034286  
1599839      85090.0          81441.0                 85.034286  


In [ ]:
# split into 95% working dataset
# split into 5% debugging dataset

merged_df = pd.read_pickle('merged_data.pkl')

debugging_df = merged_df.sample(frac=0.05)
# dropping all the ones in the 5% so its 95% working
working_df = merged_df.drop(debugging_df.index)


debugging_df.to_pickle('debugging_dataset.pkl')
working_df.to_pickle('working_dataset.pkl')


In [24]:
import pandas as pd

merged_df = pd.read_csv('merged_data.csv')
merged_df['Date'] = pd.to_datetime(merged_df['Date'], infer_datetime_format=True)

df = pd.read_csv('ActualEnergyPrices.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
dropped = df[df['HUB'] != 'ILLINOIS.HUB']

avg_prices = dropped.groupby('Date')['LMP'].mean().reset_index()
avg_prices.rename(columns={'LMP': 'AveragePrice_OtherStates'}, inplace=True)

merged_df = merged_df.merge(avg_prices, on='Date', how='left')
merged_df = merged_df[merged_df['HUB'] == 'ILLINOIS.HUB']

merged_df.to_csv('merged_data2.csv', index=False)




FileNotFoundError: [Errno 2] No such file or directory: 'ActualEnergyPrices.csv'

In [25]:
import pandas as pd

# Load merged data from pickle instead of CSV
merged_df = pd.read_pickle('merged_data.pkl')
merged_df['Date'] = pd.to_datetime(merged_df['Date'], infer_datetime_format=True)

# Load actual energy prices
df = pd.read_csv('ActualEnergyPrices2.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')

# Drop rows where HUB is not ILLINOIS.HUB
dropped = df[df['HUB'] != 'ILLINOIS.HUB']

# Calculate average price for other states
avg_prices = dropped.groupby('Date')['LMP'].mean().reset_index()
avg_prices.rename(columns={'LMP': 'AveragePrice_OtherStates'}, inplace=True)

# Merge and filter for ILLINOIS.HUB
merged_df = merged_df.merge(avg_prices, on='Date', how='left')
merged_df = merged_df[merged_df['HUB'] == 'ILLINOIS.HUB']

# Save to CSV
merged_df.to_csv('merged_data2.csv', index=False)


        Date  actual_lmp           HUB  day_ahead_lmp  actual_load  \
0 2024-02-06       17.48  ARKANSAS.HUB          16.91      70777.0   
1 2024-02-06       17.48  ARKANSAS.HUB          25.41      70777.0   
2 2024-02-06       17.48  ARKANSAS.HUB          26.02      70777.0   
3 2024-02-06       17.48  ARKANSAS.HUB          17.50      70777.0   
4 2024-02-06       17.48  ARKANSAS.HUB          26.24      70777.0   

   forecasted_load  
0          70762.0  
1          70762.0  
2          70762.0  
3          70762.0  
4          70762.0  


In [27]:
import pandas as pd

# --- Helper function to clean data ---
def clean_data(df_path):
    df = pd.read_csv(df_path)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
    return df

# --- Step 1: Load all datasets ---
actual_prices = clean_data('ActualEnergyPrices2.csv')
day_ahead = clean_data('DayAheadEnergyPrice2.csv')
actual_load = clean_data('ActualLoad2.csv')
forecasted_load = clean_data('ForecastedLoad2.csv')

# --- Step 2: Separate Illinois HUB and non-Illinois HUBs ---
illinois_prices = actual_prices[actual_prices['HUB'] == 'ILLINOIS.HUB'].copy()
other_prices = actual_prices[actual_prices['HUB'] != 'ILLINOIS.HUB'].copy()

# --- Step 3: Compute average LMP of non-Illinois HUBs per hour ---
avg_other = (
    other_prices.groupby('Date')['LMP']
    .mean()
    .reset_index()
    .rename(columns={'LMP': 'AveragePrice_OtherStates'})
)

# --- Step 4: Merge Illinois HUB actual prices with average of others ---
merged_df = pd.merge(illinois_prices, avg_other, on='Date', how='left')

# --- Step 5: Merge with Day-Ahead, Actual Load, and Forecasted Load ---
dfs_to_merge = [day_ahead, actual_load, forecasted_load]

for df in dfs_to_merge:
    merged_df = pd.merge(merged_df, df, on='Date', how='left')

# --- Step 6: Rename columns for clarity ---
merged_df.rename(
    columns={
        'LMP': 'actual_lmp',
        'lmp': 'day_ahead_lmp',
        'Load_x': 'actual_load',
        'Load_y': 'forecasted_load'
    },
    inplace=True
)

# --- Step 7: Round timestamps to the hour and drop duplicates ---
merged_df['Date'] = merged_df['Date'].dt.floor('H')
merged_df = merged_df.drop_duplicates(subset='Date', keep='first')

# --- Step 8: Drop unnecessary columns ---
for col in ['node', 'HUB']:
    if col in merged_df.columns:
        merged_df.drop(columns=[col], inplace=True)

# --- Step 9: Save results ---
merged_df.to_csv('merged_data2.csv', index=False)
merged_df.to_pickle('merged_data.pkl')

print("✅ Merged successfully: one row per hour, Illinois only, with average of other states included.")


✅ Merged successfully: one row per hour, Illinois only, with average of other states included.


In [3]:
!pip install meteostat

In [28]:
import pandas as pd
from datetime import datetime
from meteostat import Point, Hourly

merged_df = pd.read_csv('merged_data2.csv')

merged_df['Date'] = pd.to_datetime(merged_df['Date'])

start_date = merged_df['Date'].min()
end_date = merged_df['Date'].max()

# 3. Define the location (Chicago O'Hare Airport)
# Latitude, Longitude, Altitude for Chicago
location = Point(41.9742, -87.9073, 205)

# 4. Fetch the hourly weather data
print(f"Fetching weather data for Chicago O'Hare from {start_date} to {end_date}...")
weather_data = Hourly(location, start_date, end_date).fetch()
print("Weather data fetched successfully.")

# Select and process weather features
weather_features = weather_data[['temp', 'wspd', 'coco']].copy()
weather_features.rename(columns={'temp': 'temperature_celsius', 'wspd': 'wind_speed_kmh'}, inplace=True)
weather_features['coco'].fillna(method='ffill', inplace=True)

# --- RENAMING LOGIC STARTS HERE ---

# 1. Define the human-readable map for coco codes
coco_map = {
    1: 'Clear', 2: 'Fair', 3: 'Cloudy', 4: 'Overcast', 5: 'Fog',
    6: 'Freezing_Fog', 7: 'Light_Rain', 8: 'Rain', 9: 'Heavy_Rain',
    10: 'Freezing_Rain', 11: 'Heavy_Freezing_Rain', 12: 'Sleet',
    13: 'Heavy_Sleet', 14: 'Light_Snow', 15: 'Snow', 16: 'Heavy_Snow',
    17: 'Rain_Shower', 18: 'Heavy_Rain_Shower', 19: 'Sleet_Shower',
    20: 'Heavy_Sleet_Shower', 21: 'Snow_Shower', 22: 'Heavy_Snow_Shower',
    23: 'Lightning', 24: 'Hail', 25: 'Thunderstorm',
    26: 'Heavy_Thunderstorm', 27: 'Storm'
}
# Note: I replaced spaces with underscores for cleaner column names.

# 2. Perform One-Hot Encoding as before
coco_dummies = pd.get_dummies(weather_features['coco'], prefix='coco')

# 3. Create a dictionary to map the old column names to the new ones
# e.g., {'coco_1.0': 'coco_Clear', 'coco_7.0': 'coco_Light_Rain'}
new_column_names = {}
for col in coco_dummies.columns:
    # Extract the code number (e.g., '1.0' from 'coco_1.0')
    code_str = col.split('_')[1]
    code_int = int(float(code_str))

    # Find the description in the map, default to the code if not found
    description = coco_map.get(code_int, f'Unknown_{code_int}')

    # Store the mapping
    new_column_names[col] = f'coco_{description}'

# 4. Rename the columns of the dummies DataFrame
coco_dummies.rename(columns=new_column_names, inplace=True)

# --- RENAMING LOGIC ENDS HERE ---



# Join the new one-hot encoded columns back to your weather features
weather_features = weather_features.join(coco_dummies)

# Drop the original 'coco' column as it's now redundant
weather_features.drop('coco', axis=1, inplace=True)



# 6. Merge the weather data with your original DataFrame
# We merge based on the 'Date' column and the index of the weather data
merged_df = pd.merge(merged_df, weather_features, left_on='Date', right_index=True, how='left')

bool_cols = merged_df.select_dtypes(include='bool').columns
# --- Explicit Conversion: The Best Practice ---
# Convert any boolean columns to integers (0 and 1)
merged_df[bool_cols] = merged_df[bool_cols].astype(int)

# Display the first few rows of the final, merged DataFrame
print("\nPreview of the merged DataFrame with weather data:")
print(merged_df.head())

Fetching weather data for Chicago O'Hare from 2022-01-01 00:00:00 to 2025-02-05 18:00:00...
Weather data fetched successfully.

Preview of the merged DataFrame with weather data:
                 Date  actual_lmp  AveragePrice_OtherStates  day_ahead_lmp  \
0 2022-01-01 00:00:00       22.46                 25.725714          29.54   
1 2022-01-01 02:00:00       22.54                 23.585714          26.32   
2 2022-01-01 03:00:00       21.82                 22.627143          23.81   
3 2022-01-01 04:00:00       21.43                 22.382857          22.58   
4 2022-01-01 05:00:00       20.69                 21.810000          22.60   

   actual_load  forecasted_load  temperature_celsius  wind_speed_kmh  \
0      67114.0            68692                  5.6            14.8   
1      63696.0            62585                  5.0            14.8   
2      62627.0            61348                  4.4            24.1   
3      62217.0            60685                  4.4            

The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [29]:
# Get the counts of each unique value in the 'coco' column
coco_counts = weather_data['coco'].value_counts()

# Also get the percentage of total time
coco_percentage = weather_data['coco'].value_counts(normalize=True) * 100

# Create a DataFrame for a nice, clean presentation
coco_df = pd.DataFrame({
    'Hourly_Count': coco_counts,
    'Percentage_of_Year': coco_percentage
})
coco_df.index.name = 'Coco_Code'

# Create a map for human-readable descriptions (based on Meteostat documentation)
coco_map = {
    1: 'Clear', 2: 'Fair', 3: 'Cloudy', 4: 'Overcast', 5: 'Fog',
    6: 'Freezing Fog', 7: 'Light Rain', 8: 'Rain', 9: 'Heavy Rain',
    10: 'Freezing Rain', 11: 'Heavy Freezing Rain', 12: 'Sleet',
    13: 'Heavy Sleet', 14: 'Light Snow', 15: 'Snow', 16: 'Heavy Snow',
    17: 'Rain Shower', 18: 'Heavy Rain Shower', 19: 'Sleet Shower',
    20: 'Heavy Sleet Shower', 21: 'Snow Shower', 22: 'Heavy Snow Shower',
    23: 'Lightning', 24: 'Hail', 25: 'Thunderstorm',
    26: 'Heavy Thunderstorm', 27: 'Storm'
}

# Add the descriptions to our results DataFrame
coco_df['Description'] = coco_df.index.map(coco_map)

print("\nNumber and Percentage of Occurrences for each Weather Condition Code:\n")
print(coco_df)


Number and Percentage of Occurrences for each Weather Condition Code:

           Hourly_Count  Percentage_of_Year        Description
Coco_Code                                                     
2.0                8100            30.10369               Fair
3.0                6647           24.703609             Cloudy
4.0                6372            23.68157           Overcast
1.0                1563            5.808897              Clear
7.0                1457            5.414948         Light Rain
14.0                774            2.876575         Light Snow
5.0                 741             2.75393                Fog
8.0                 737            2.739064               Rain
9.0                 279            1.036905         Heavy Rain
18.0                 69            0.256439  Heavy Rain Shower
12.0                 56            0.208124              Sleet
15.0                 53            0.196975               Snow
13.0                 38            0.141227   

In [19]:
# # --- Step 2: Analyze and Rank the Weather Conditions ---
# # This is the part you should run on your actual DataFrame.

# print("--- Analysis of Weather Conditions vs. Average LMP ---\n")

# # Find all columns that start with 'coco_'
# coco_columns = [col for col in merged_df.columns if col.startswith('coco_')]

# # Dictionary to store the results
# results = {}

# # Loop through each weather condition column
# for condition_col in coco_columns:
#     # Filter the DataFrame for rows where this condition is True (or 1)
#     when_condition_occurs = merged_df[merged_df[condition_col] == 1]

#     # Calculate the mean LMP for those hours
#     mean_lmp = when_condition_occurs['actual_lmp'].mean()

#     # Count how many times it occurred
#     count = len(when_condition_occurs)

#     # Store the results, extracting the clean condition name
#     clean_name = condition_col.replace('coco_', '')
#     results[clean_name] = {'Mean_LMP': mean_lmp, 'Hourly_Count': count}

# # Convert the results to a DataFrame for sorting and display
# results_df = pd.DataFrame.from_dict(results, orient='index')
# results_df.sort_values(by='Mean_LMP', ascending=False, inplace=True)

# print("Weather Conditions Ordered by their Average 'actual_lmp':\n")
# print(results_df)

--- Analysis of Weather Conditions vs. Average LMP ---

Weather Conditions Ordered by their Average 'actual_lmp':

                    Mean_LMP  Hourly_Count
Light_Snow         32.351475           122
Sleet              32.054000            10
Cloudy             30.092802          2009
Light_Rain         29.124889           407
Snow               27.915714             7
Heavy_Sleet        27.530000             3
Overcast           27.085632          2161
Rain               26.729009           232
Fair               26.253824          2526
Fog                26.210479           167
Heavy_Rain_Shower  26.171765            51
Clear              25.813891           550
Heavy_Snow         23.110000             3
Heavy_Rain         21.301290            93
Thunderstorm       21.172000            10


In [30]:
merged_df.to_csv('merged_data4.csv', index=False)